# example1

The simulation of financial markets through the use of agent-based models is an increasingly popular technique to understand the microstructure of their dynamics from the bottom up. In this notebook, we'll look at an example market simulation.

In [2]:
# Usual imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

Looking at this paper: https://www.nature.com/articles/srep08399.pdf ...

In [14]:
class agentens:
    def __init__(self, setup : dict):
        """A class for an ensemble of agents which
        can be evolved in time."""
        self.setup = setup
        self._stocks_held = None
        self._invest_horizons = None
        self._max_horizon = None
        self._Nhist = None
        self._price_diff_history = None
        self.ln_returns = np.zeros(
            (self.setup["Nagents"], self.setup["Nreals"])
        )
    @property
    def stocks_held(self):
        if self._stocks_held is None:
            self._stocks_held = np.random.randint(
                0,
                self.setup["Nstocks"] + 1,
                size=(
                    self.setup["Nagents"],
                    self.setup["Nreals"],
                ),
            )
        return self._stocks_held
    @property
    def invest_horizons(self):
        if self._invest_horizons is None:
            self._invest_horizons = np.random.exponential(
                np.tensordot(
                    1.0 
                    / np.random.gamma(
                        self.setup["GammaShape"],
                        self.setup["GammaScale"],
                        size=self.setup["Nagents"],
                    ),
                    np.ones(self.setup["Nreals"]),
                    axes=0,
                ),
                size=(
                    self.setup["Nagents"],
                    self.setup["Nreals"],
                )
            )
        return self._invest_horizons
    @property
    def max_horizon(self):
        if self._max_horizon is None:
            self._max_horizon = self.invest_horizons.max()
        return self._max_horizon
    @property
    def Nhist(self):
        if self._Nhist is None:
            self._Nhist = int(self.max_horizon / self.setup["tstep"])
        return self._Nhist
    @property
    def price_diff_history(self):
        if self._price_diff_history is None:
            self._price_diff_history = np.zeros(
                (self.Nhist, self.setup["Nstocks"], self.setup["Nreals"])
            )
        return self._price_diff_history    
    def iterate(self, prices : np.ndarray):
        """Take the ensemble a step forward in time by
        asking each agent to make a buy-sell-hold decision."""
        draws = np.random.uniform(size=(self.setup["Nagents"], self.setup["Nreals"]))
        self.decisions = (
            0
            + 1 * (self.buyprobs > draws)
            - 1 * (self.sellprobs + self.buyprobs > draws >= self.buyprobs)
        )
        self.ln_returns = np.cumsum(np.log(self.price_diff_history), axis=0)

In [16]:
class marketens:
    def __init__(self, setup : dict):
        """A class for an ensemble of simulated market 
        prices which can be evolved in time."""
        self.setup = setup
        self.prices = 10.0 * np.random.uniform(
            size=(self.setup["Nstocks"], self.setup["Nreals"])
        )
        self._ae = None
    @property
    def ae(self):
        if self._ae is None:
            self._ae = agentens(self.setup)
        return self._ae
    def iterate(self, prices : np.ndarray) -> np.ndarray:
        """Take the market prices (and the ensemble) 
        a step forward in time."""
        new_prices = ###
        return new_prices

In [25]:
k = 100.0
setup = {
    "Nstocks" : 100,
    "Nagents" : 1000,
    "Nreals" : 10,
    "GammaShape" : k,
    "GammaScale" : 1.0 / k,
    "tstep" : 0.1,
}
me = marketens(setup)

In [26]:
me.ae.max_horizon

10.725685626262921